In [ ]:
import astropy
import astroquery
import matplotlib.pyplot as plt
import pandas
import ipywidgets as widgets
from astropy.coordinates import AltAz, EarthLocation, SkyCoord
from astropy import units as u
from astropy.time import Time, TimeDelta
from datetime import datetime
from ipywidgets import Layout, Button, Box, VBox, HBox
from IPython.display import display, clear_output
from astroplan import FixedTarget, Observer
from astroplan.plots import plot_airmass, plot_finder_image, plot_sky
from astroquery.simbad import Simbad
import astropy.coordinates as coordinates
import warnings
warnings.filterwarnings("ignore", message="Numerical value without unit or explicit format passed to TimeDelta, assuming days")

In [ ]:
Simbad.add_votable_fields("U", "V", "B")

In [ ]:
from astropy.utils import iers
iers.conf.IERS_A_URL = 'ftp://cddis.gsfc.nasa.gov/pub/products/iers/finals2000A.all'
iers.conf.IERS_A_URL_MIRROR = 'https://datacenter.iers.org/data/9/finals2000A.all'
from astroplan import download_IERS_A
download_IERS_A()

In [ ]:
# Define observer (RHO)
RHO = Observer(
    location=coordinates.EarthLocation(lat=29.4001, lon=-82.5862*u.deg, height=23*u.m),
    timezone='US/Eastern',
    name='Rosemary Hill Observatory'
)
now = Time.now()

In [ ]:
# Alignment stars
# TODO: add optional date + time fields so someone can Plan

# List of possible alignment stars - can be added to if desired. 
# Currently organized by brightest mag V to dimmest
alignment_stars = ['Sirius', 'Antares', 'Arcturus', 'Vega', 'Capella', 
                   'Procyon', 'Altair', 'Aldebaran', 'Spica', 'Fomalhaut', 
                   'Deneb', 'Regulus', 'Dubhe', 'Mirfak', 'Polaris', 'Schedar']

# Style prevents shortening of description
style = {"description_width": "initial"}

# Define dropdown
alignment_stars_select = widgets.Dropdown(
    options=[],
    value=None,
    description="Alignment star:",
    style=style
)
output = widgets.Output()

# Determines which alignment stars are up to add this info to the dropdown menu
def determine_up_stars():
    now = Time.now()                                # Update time
    stars_list = []                                 # List of dropdown options

    for star in alignment_stars:
        curr_target = FixedTarget(coordinates.SkyCoord.from_name(star), name=star)
        if RHO.target_is_up(now, curr_target):
            stars_list.append(star + " (Up)")       # So user can see if a given align star is in the sky
        else:
            stars_list.append(star)
    alignment_stars_select.options = stars_list         # Update dropdown options

# Eventhandler for dropdown menu that spits out info about a given align star
def dropdown_align_star_eventhandler(star):
    now = Time.now()                                # Update time
    clear_output(wait=True)         # Makes it so the output of the last query is cleared
    determine_up_stars()            # Refresh list of stars

    name = alignment_stars_select.value # Grabs the selected star name

    if "(Up)" in name:              # Cuts off the (Up) part of the name if the star is indeed up, so SIMBAD can query
        name = name[0:-5]

    # SIMBAD shenanigans to get some relevant info and convert it to hmsdms bc SIMBAD doesn't do that natively anymore???
    result_table = Simbad.query_object(name)[["main_id", "ra", "dec", "V"]]
    coords = SkyCoord(ra=result_table["ra"], dec=result_table["dec"])
    info = [result_table["main_id"][0], coords.to_string('hmsdms'), result_table["V"][0]]
    curr_target = FixedTarget(coords, name=name)
    rise_set = [eastern(RHO.target_rise_time(time=now, target=curr_target)), eastern(RHO.target_set_time(time=now, target=curr_target))]
    alt_az = coords.transform_to(AltAz(obstime=now, location=RHO.location))
    str_alt = str(alt_az.alt)[1:-8] + "s"
    str_az = str(alt_az.az)[1:-8] + "s"

    # Redisplay dropdown
    display(dis)
    # Print relevant info
    print("Name: " + info[0])
    print("Coordinates: " + str(info[1])[2:13] +", " + str(info[1])[22:33])         # Cutting off the long decimal points for readibility w/o rounding - we don't need to be THAT precise for calib stars
    print("Magnitude V: " + str(round(float(info[2]), 5)))
    print("Rises: " + rise_set[0] + " EST")
    print("Sets: " + rise_set[1] + " EST")     
    print("Altitude: " + str_alt)
    print("Azimuth: " + str_az)

    # Plot finder image
    ax, hdu = plot_finder_image(curr_target, fov_radius=15*u.arcmin)
    title = "Finder image for " + name
    ax.set_title(title); 

# Init dropdown list of stars for the first time
determine_up_stars()

# Idk what this does but I THINK it's just the host for the widget to not close maybe
alignment_stars_select.observe(dropdown_align_star_eventhandler, names='value')

# Define and display dropdown
dis = widgets.VBox([alignment_stars_select])
display(dis)

# Used https://towardsdatascience.com/interactive-data-analysis-with-dropdown-menu-ipywidgets-and-plotly-in-jupyter-notebook-591a84a81b22 
# for reference in doing the dropdown selection
